##  Regularized Discriminant Analysis

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import warnings

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [4]:
csv_df = pd.read_csv('C:/ME Lethal Prediction/ME_FNL1.csv')
csv_df.head()

,Decade,Year,Month,Week,Day,Extended,Country,Success,Multiple,Suicide,...,ArmedAssault,Infrastructure,Assassination,UnknownAttack,Explosives,UnknownWeapon,Firearms,Incendiary,Melee,OtherWeapon
0,1970,1970,3,2,14,0,Egypt,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1970,1970,3,4,29,0,Lebanon,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1970,1970,4,3,15,0,OtherCountry,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1970,1970,4,4,25,0,Turkey,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1970,1970,6,1,7,1,OtherCountry,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
# define x and y
feature_cols = ['OtherProvince',
       'BeirutProvince', 'IstanbulProvince', 'AleppoProvince',
       'WestBankProvince', 'BaghdadProvince', 'SouthernProvince',
       'BenghaziProvince', 'NinevehProvince', 'SaladinProvince',
       'AlAnbarProvince', 'DiyalaProvince', 'KirkukProvince', 'BabilProvince',
       'NorthSinaiProvince', 'OtherCity', 'BeirutCity', 'Bomb',
       'HostageKidnap', 'OtherAttack', 'ArmedAssault', 'Infrastructure',
       'Assassination']

X = csv_df[feature_cols]
Y = csv_df.Lethal

# split X and Y into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y, shuffle = True, random_state = 66)

Since we are charged with creating the best model possible, let us create new features.  Here, we’ll create second order polynomials and interaction terms, and separately, we create third order polynomials and third degree interaction terms. Creation of these terms will bring up some issues but will spend more time on that in a bit.  For now, just notice that creating third order polynomials increased our column count from 23 to 1,539. 

In [7]:
# Create interaction terms (interaction of each regressor pair + polynomial)
#Interaction terms need to be created in both the test and train datasets
interaction2 = PolynomialFeatures(degree = 2, include_bias = False, interaction_only = False, order = 'C') #second degree
interaction3 = PolynomialFeatures(degree = 3, include_bias = False, interaction_only = False, order = 'C') #third degree

#traning
X_train_2 = pd.DataFrame(interaction2.fit_transform(X_train), columns = interaction2.get_feature_names(input_features = X_train.columns))
X_train_3 = pd.DataFrame(interaction3.fit_transform(X_train), columns = interaction3.get_feature_names(input_features = X_train.columns))

X_train_2.head()

#test
X_test_2 = pd.DataFrame(interaction2.fit_transform(X_test), columns = interaction2.get_feature_names(input_features = X_train.columns))
X_test_3 = pd.DataFrame(interaction3.fit_transform(X_test), columns = interaction3.get_feature_names(input_features = X_train.columns))

X_test_3.head()

C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\li

,OtherProvince,BeirutProvince,IstanbulProvince,AleppoProvince,WestBankProvince,BaghdadProvince,SouthernProvince,BenghaziProvince,NinevehProvince,SaladinProvince,...,ArmedAssault^3,ArmedAssault^2 Infrastructure,ArmedAssault^2 Assassination,ArmedAssault Infrastructure^2,ArmedAssault Infrastructure Assassination,ArmedAssault Assassination^2,Infrastructure^3,Infrastructure^2 Assassination,Infrastructure Assassination^2,Assassination^3
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since we created interaction terms and polynomials, multicollinearity will certainly be an issue.  Here, we check if multicollinearity exists in the original data set, and then we go through the newly created two data sets containing second and third degree polynomials the same way.  We make use of VIF and identify all variables with a VIF of greater than 5.  We simply will eliminate these variables from the analysis. 

In [9]:
################################
## Deal with multicollinearity
################################

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1st order polynomial ######################

x_temp_train1 = sm.add_constant(X_train)
vif_train1 = pd.DataFrame()
vif_train1["VIF Factor"] = [variance_inflation_factor(x_temp_train1.values, i) for i in range(x_temp_train1.values.shape[1])]
vif_train1["features"] = x_temp_train1.columns
pd.set_option('display.max_rows', 300)
print(vif_train1.round(1))

vif_train1_a = vif_train1[vif_train1["VIF Factor"] < 5.0] #identify all variables wit VIF less then 5 and keep
#print(vif2.round(1))
feat_list = vif_train1_a["features"].tolist()  #save desired features to list
feat_list.remove(feat_list[0])
print(feat_list)

X_train = X_train[feat_list] #keep features on feature list only, drop all other features for train
X_test = X_test[feat_list]   #keep features on feature list only, drop all other features for test

C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


    VIF Factor            features
0          0.0               const
1          inf       OtherProvince
2          inf      BeirutProvince
3          inf    IstanbulProvince
4          inf      AleppoProvince
5          inf    WestBankProvince
6          inf     BaghdadProvince
7          inf    SouthernProvince
8          inf    BenghaziProvince
9          inf     NinevehProvince
10         inf     SaladinProvince
11         inf     AlAnbarProvince
12         inf      DiyalaProvince
13         inf      KirkukProvince
14         inf       BabilProvince
15         inf  NorthSinaiProvince
16         5.7           OtherCity
17        43.5          BeirutCity
18        10.6                Bomb
19         3.1       HostageKidnap
20         1.3         OtherAttack
21         6.8        ArmedAssault
22         1.9      Infrastructure
23         4.4       Assassination
['HostageKidnap', 'OtherAttack', 'Infrastructure', 'Assassination']


Where the VIF Factor is inf or infinite, there is perfect collinearity and such variables definitely need to be removed.

In [10]:
# 2nd order polynomial ####################

x_temp_train2 = sm.add_constant(X_train_2)
vif_train2 = pd.DataFrame()
vif_train2["VIF Factor"] = [variance_inflation_factor(x_temp_train2.values, i) for i in range(x_temp_train2.values.shape[1])]
vif_train2["features"] = x_temp_train2.columns
pd.set_option('display.max_rows', 300)
#print(vif_train1.round(1))

vif_train2_a = vif_train2[vif_train2["VIF Factor"] < 5.0]
#print(vif2.round(1))

feat_list2 = vif_train2_a["features"].tolist()
feat_list2.remove(feat_list2[0])
print(feat_list2)

X_train_2 = X_train_2[feat_list2] #keep features on feature list only, drop all other features for train
X_test_2 = X_test_2[feat_list2]   #keep features on feature list only, drop all other features for test
X_test_2

C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


[]


""
0
1
2
3
4
...
8937
8938
8939
8940


In [ ]:
# 3rd order polynomial ####################

x_temp_train3 = sm.add_constant(X_train_3)
vif_train3 = pd.DataFrame()
vif_train3["VIF Factor"] = [variance_inflation_factor(x_temp_train3.values, i) for i in range(x_temp_train3.values.shape[1])]
vif_train3["features"] = x_temp_train3.columns
pd.set_option('display.max_rows', 300)
#print(vif_train3.round(1))

vif_train3_a = vif_train3[vif_train3["VIF Factor"] < 5.0]
#print(vif3.round(1))

feat_list3 = vif_train3_a["features"].tolist()
feat_list3.remove(feat_list3[0])
print(feat_list3)

X_train_3 = X_train_3[feat_list3] #keep features on feature list only, drop all other features for train
X_test_3 = X_test_3[feat_list3]   #keep features on feature list only, drop all other features for test

C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Nicholas\anaconda3\envs\ME_Lethal_Terrorism\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
# Default LDA model without any tuning - base metric

LDA_model_default = LinearDiscriminantAnalysis()
LDA_model_default.fit(X_train, Y_train)
y_pred_LDA_default = LDA_model_default.predict(X_test)

In [ ]:
# Parameter tuning with GridSearchCV 

#######################
### LDA  - lsqr & eigen
#######################

estimator_1 = LinearDiscriminantAnalysis(shrinkage = 'auto')
parameters_1 = {
    'solver': ('lsqr','eigen'),  # note svd does not run with shrinkage and models using it will be tuned separately
    'n_components': (1, 5, 1),
                   }
# with GridSearch
grid_search_lda_A = GridSearchCV(
    estimator = estimator_1,
    param_grid = parameters_1,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
lda_A1 = grid_search_lda_A.fit(X_train, Y_train)
y_pred_1 = lda_A1.predict(X_test)

lda_A2 = grid_search_lda_A.fit(X_train_2, Y_train)
y_pred_2 = lda_A2.predict(X_test_2)

lda_A3 = grid_search_lda_A.fit(X_train_3, Y_train)
y_pred_3 = lda_A3.predict(X_test_3)

In [ ]:
# Parameter tuning with GridSearchCV 

#############
### LDA - svd
#############

estimator_2 = LinearDiscriminantAnalysis(solver = 'svd', )#note svd does not run with shrinkage and models using it will be tuned separately
parameters_2 = {
    'n_components': (0, 5, 1),
    'store_covariance' :(True, False),
                   }
# with GridSearch
grid_search_lda_B = GridSearchCV(
    estimator = estimator_2,
    param_grid = parameters_2,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
lda_B1 = grid_search_lda_B.fit(X_train, Y_train)
y_pred_4 = lda_B1.predict(X_test)

lda_B2 = grid_search_lda_B.fit(X_train_2, Y_train)
y_pred_5 = lda_B2.predict(X_test_2)

lda_B3 = grid_search_lda_B.fit(X_train_3, Y_train)
y_pred_6 = lda_B3.predict(X_test_3)

Quadratic Discriminant Analysis:

The next section is focused on QDA.  A list of tunable parameters is available by clicking here. GridSearchCV was once again used for parameter tuning, and the final exercise looked at the tuning of three parameters. 

In [ ]:
# Base QDA Without any tuning
QDA_model_default = QuadraticDiscriminantAnalysis()
QDA_model_default.fit(X_train, Y_train)
y_pred_QDA_default = QDA_model_default.predict(X_test)

# Parameter tuning with GridSearchCV 

#######
### QDA
#######

estimator_3 = QuadraticDiscriminantAnalysis()
parameters_3 = {
    'reg_param': (0.00001, 0.0001, 0.001,0.01, 0.1), 
    'store_covariance': (True, False),
    'tol': (0.0001, 0.001,0.01, 0.1), 
                   }
# with GridSearch
grid_search_qda = GridSearchCV(
    estimator = estimator_3,
    param_grid = parameters_3,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
qda_1 = grid_search_qda.fit(X_train, y_train)
y_pred_7 = qda_1.predict(X_test)

qda_2 = grid_search_qda.fit(X_train_2, y_train)
y_pred_8 = qda_2.predict(X_test_2)

qda_3 = grid_search_qda.fit(X_train_3, y_train)
y_pred_9 = qda_3.predict(X_test_3)